In [232]:
import numpy as np
import pandas as pd
import random
from sklearn import datasets
import matplotlib.pyplot as plt

iris = datasets.load_iris()
X = np.array(iris.data)
y = np.array(iris.target).reshape(X.shape[0], 1)

df = np.concatenate((X, y), axis = 1)
df = np.random.permutation(df)

n_total = df.shape[0]

one_hot = np.zeros([n_total, 3])

for idx, val in enumerate(df[:, -1]):
    one_hot[idx][int(val)] = 1

# Split the data
X_train = df[0:int(0.8*n_total), :-1].astype(np.float64)
y_train = one_hot[0:int(0.8*n_total)].astype(np.float64)

X_test = df[int(0.8*n_total):, :-1].astype(np.float64)
y_test = df[int(0.8*n_total):, -1].astype(np.float64)

In [233]:
class NeuralNetworks(object):
    def __init__(self, n_features = 4, n_hidden = 6, n_output = 3, iterations = 1000, alpha = 0.001):
        self.n_features = n_features
        self.n_hidden = n_hidden
        self.n_output = n_output  
        self.iterations = iterations
        self.alpha = alpha
    
    def sig_func(self, z):
        return 1 / (1 + np.exp(-z))
    
    # Need to randomly initialize theta. Zeros will cause repeatability.
    def init_theta(self):
        theta1 = np.random.uniform(-1, 1, size = self.n_hidden * (self.n_features + 1) )\
        .reshape(self.n_hidden, (self.n_features + 1) )
        theta2 = np.random.uniform(-1, 1, size = self.n_output * (self.n_hidden + 1) )\
        .reshape(self.n_output, (self.n_hidden + 1) )
        return theta1, theta2
    
    def forward_prop(self, X, theta1, theta2):        
        a1 = np.insert(X, 0, 1, axis = 1)
        z2 = theta1.dot(a1.T)
        a2 = self.sig_func(z2)
        a2 = np.insert(a2, 0, 1, axis = 0)
        z3 = theta2.dot(a2)
        a3 = self.sig_func(z3)
        return a1, a2, a3
        
    def cost_func(self, a3, X, y):         
        J = (-1 / X.shape[0]) * ( (y * np.log(a3.T)) +(1 - y) * np.log(1 - a3.T) ).sum()
        return J
           
    def back_prop(self, a1, a2, a3, theta2, y):
        
        small_delta3 = a3 - y.T
        small_delta2 = theta2.T.dot(small_delta3) * (a2* (1 - a2))
        # Remove bias
        small_delta2 = small_delta2[1:, :]
        
        big_delta1 = small_delta2.dot(a1)
        big_delta2 = small_delta3.dot(a2.T)
        return big_delta1, big_delta2
    
    def fit(self, X, y):
        theta1, theta2 = self.init_theta()
        
        for iteration in range(self.iterations):
            a1, a2, a3 = self.forward_prop(X, theta1, theta2)
            
            J = self.cost_func(a3, X, y)
           
            big_delta1, big_delta2 = self.back_prop(a1, a2, a3, theta2, y)
            # Update theta
            theta1 -= self.alpha * big_delta1
            theta2 -= self.alpha * big_delta2
            return theta1, theta2
        
    def predict(self, X, theta1, theta2):        
        a1, a2, a3 = self.forward_prop(X, theta1, theta2)
        y_pred = np.argmax(a3, axis = 0)
        return y_pred

In [234]:
NN = NeuralNetworks()

final_theta1, final_theta2 = NN.fit(X_train, y_train)

prediction = NN.predict(X_test, final_theta1, final_theta2)

print('accuracy: ' + str( np.sum(y_test == prediction) / len(y_test) ))

accuracy: 0.3333333333333333
